# Example 101 Thermochemistry using DFTB+

Clone repo on GitHub: https://github.com/MolarVerse/ThermoScreening and install using `pip install .` in the root directory.

In [ ]:
from ThermoScreening.thermo.api import dftbplus_thermo
from ThermoScreening.calculator import Geoopt, Hessian, Modes

from ase.io import read, write

import os

skf_dir = 'skf-files/3ob-3-1'

# set DFTB perfix and path
os.environ['DFTB_PREFIX'] = os.path.join(os.environ['HOME'], skf_dir)
os.environ['OMP_NUM_THREADS'] = '6'

DFTB+ parameters for thermochemistry with COSMO solvation model

In [ ]:
dftb_parameters = dict(
    # SCC
    Hamiltonian_SCC="Yes",
    Hamiltonian_MaxSCCIterations=250,
    Hamiltonian_SCCTolerance='1.0e-6',
    Hamiltonian_ReadInitialCharges="No",
    Hamiltonian_ShellResolvedScc="No",

    # Fermi smearing
    Hamiltonian_Filling="Fermi {",
    Hamiltonian_Filling_empty="Temperature [Kelvin] = 300",

    # Convergence helper
    Hamiltonian_Mixer="DIIS{}",

    # Dispersion correction
    Hamiltonian_Dispersion="DftD4 {",
    Hamiltonian_Dispersion_s6=1.0,
    Hamiltonian_Dispersion_s8=0.4727337,
    Hamiltonian_Dispersion_s9=0.0,
    Hamiltonian_Dispersion_a1=0.5467502,
    Hamiltonian_Dispersion_a2=4.4955068,

    # H-Damping
    Hamiltonian_HCorrection="Damping {",
    Hamiltonian_HCorrection_Exponent=4.00,

    # Halogen correction
    Hamiltonian_HalogenXCorr="Yes",

    # Are guessed by ase
    Hamiltonian_MaxAngularMomentum_="",

    # Solvation model
    Hamiltonian_Solvation_="Cosmo",
    Hamiltonian_Solvation_Radii="VanDerWaalsRadiiD3 {}",
    Hamiltonian_Solvation_RadiiScaling=1.55,
    Hamiltonian_Solvation_AngularGrid=110,
    Hamiltonian_Solvation_Solver="DomainDecomposition{",
    Hamiltonian_Solvation_Solver_MaxMoment=10,
    Hamiltonian_Solvation_Solver_Accuracy=1.0e-8,
    Hamiltonian_Solvation_Solver_Regularisation=0.2,
    Hamiltonian_Solvation_Solvent="""FromConstants{
        Epsilon = 37.0,
        MolecularMass [amu] = 73.1,
        Density [kg/l] = 0.95
      }""",
    Hamiltonian_Solvation_empty="FreeEnergyShift [kcal/mol] = 0.0",

    # Hubbard derivatives
    Hamiltonian_ThirdOrderFull="Yes",
    Hamiltonian_hubbardderivs_="",
    Hamiltonian_hubbardderivs_C=-0.1492,
    Hamiltonian_hubbardderivs_N=-0.1535,
    Hamiltonian_hubbardderivs_O=-0.1575,
    Hamiltonian_hubbardderivs_H=-0.1857,
    Hamiltonian_hubbardderivs_S=-0.11,
    Hamiltonian_hubbardderivs_P=-0.14,
    Hamiltonian_hubbardderivs_F=-0.1623,
    Hamiltonian_hubbardderivs_Cl=-0.0697,
    Hamiltonian_hubbardderivs_Br=-0.0573,
    Hamiltonian_hubbardderivs_I=-0.0433,
    Hamiltonian_hubbardderivs_Zn=-0.03,
    Hamiltonian_hubbardderivs_Mg=-0.02,
    Hamiltonian_hubbardderivs_Ca=-0.0340,
    Hamiltonian_hubbardderivs_K=-0.0339,
    Hamiltonian_hubbardderivs_Na=-0.0454,

    # Analysis
    Analysis_="",
    Analysis_CalculateForces="Yes",
    Analysis_MullikenAnalysis="Yes",

    # ParserVersion
    ParserOptions_ParserVersion=12,
)

Create `ase.Atoms` from SMILES

In [ ]:
from wfl.generate import smiles

# quinone smile
smile = 'C1=CC(=O)C=CC1=O'
system = smiles.smi_to_atoms(smile)


--------

Run ThermoScreening with DFTB+:

In [ ]:
thermo = dftbplus_thermo(system, charge=0, **dftb_parameters)
zero = thermo.total_EeGtot()

In [ ]:
thermo = dftbplus_thermo(system, charge=-1, **dftb_parameters)
neg1 = thermo.total_EeGtot()

In [ ]:
thermo = dftbplus_thermo(system, charge=-2, **dftb_parameters)
neg2 = thermo.total_EeGtot()

In [ ]:
print(zero, neg1, neg2)

Calculate redox potential:

In [ ]:
# faraday constant
F = 96485.3329
# hartree to j/mol
hartree = 2625500.2

redox1 = -(neg1 - zero) * hartree / F
redox2 = -(neg2 - neg1) * hartree / F

print(f'{smile} redox1: {redox1} V')
print(f'{smile} redox2: {redox2} V')